# Prediction on mean HR (every 2s)

In [2]:
import numpy as np
import pandas as pd
import os
import glob

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('seaborn-v0_8-whitegrid')
# %matplotlib qt

In [5]:
# import sys
# sys.path.append("../")

processed_data_path = "/Users/augenpro/Documents/Empatica/data_sara/data/GGIR_input/"

start_end_sleep = np.load(processed_data_path + "SPT_window_GGIR.npy", allow_pickle=True)
ppg_df = pd.read_parquet(processed_data_path + "ppg.parquet")
temp_df = pd.read_parquet(processed_data_path + "temp.parquet")
acc_df = pd.read_parquet(processed_data_path + "acc.parquet")

In [6]:
# I need to divide it into portions when the device was 

t_empty_end = acc_df.index[acc_df.index.to_series().diff().dt.total_seconds() > 1]
t_empty_start = acc_df.index[np.where(acc_df.index.to_series().diff().dt.total_seconds() > 1)[0]-1]
t_empty = pd.DataFrame({"start": t_empty_start, "end": t_empty_end})

good_portions = pd.DataFrame(columns=["start", "end"])
good_portions["start"] = t_empty["end"].iloc[:-1].reset_index(drop=True)
good_portions["end"] = t_empty["start"].iloc[1:].reset_index(drop=True)
start_first_empty = t_empty["start"].iloc[0]
end_last_empty = t_empty["end"].iloc[-1]

# Segment the data into portions when the device was not in empty and perform nonwear detection
acc_df_portions = [acc_df[:start_first_empty]]
ppg_df_portions = [ppg_df[:start_first_empty]]

for i, row in good_portions.iterrows():

    if row["end"] - row["start"] < pd.Timedelta("10 min"): # if the portion is less than 10 minutes, skip it
        continue

    acc_df_portions.append(acc_df[row["start"]:row["end"]])
    ppg_df_portions.append(ppg_df[row["start"]:row["end"]])

acc_df_portions.append(acc_df[end_last_empty:])
ppg_df_portions.append(ppg_df[end_last_empty:])

### Load the model and predict

In [8]:
import sys
sys.path.append("../")
from heart_rate.beliefppg.inference.inference import infer_hr

In [9]:
hr_all = []
time_hr_all = []
for acc, ppg in zip(acc_df_portions, ppg_df_portions): ##### Each portion has the same shape between ACC and PPG lesgooo
   time = acc.index # same as ppg.index
   hr, idxs = infer_hr(ppg=ppg.values.reshape(-1,1), ppg_freq=64, acc=acc.values, acc_freq=64)
   hr_all.append(hr)
   time_hr_all.append(time[idxs])

2025-03-05 10:19:59.631982: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


660/660 [==============================] - 52s 78ms/step


2025-03-05 10:20:51.706401: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


102/102 [==============================] - 8s 73ms/step


2025-03-05 10:21:14.421090: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


171/171 [==============================] - 13s 72ms/step


2025-03-05 10:23:07.671627: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


267/267 [==============================] - 20s 72ms/step


In [ ]:
# Concatenate all the portions
hr_belief = np.concatenate(hr_all)
t_hr_belief = np.concatenate(time_hr_all)

# Convert to pandas Series
hr_belief_df = pd.Series(hr_belief, index=t_hr_belief)

((233108,), (233108,))

In [ ]:
# Save to CSV
save_data_path = "/Users/augenpro/Documents/Empatica/data_sara/data/heart_rate/"
hr_belief_df.to_csv(save_data_path + "hr_belief.csv")

### Visualization

In [3]:
import subprocess
import os

hr_file_path = "/Users/augenpro/Documents/Empatica/data_sara/data/GGIR_input/"
hr_file = os.path.join(hr_file_path, "hr_belief.pkl")
spt_file = os.path.join(hr_file_path, "SPT_window_GGIR.npy")

# This calls the script to plot the HR
script_path = os.path.abspath("../visualization/plot_hr.py")
bokeh_process = subprocess.Popen([
    "bokeh", "serve", "--show", script_path,
    "--args", "--hr_file", hr_file, "--spt_file", spt_file
])

2025-02-22 12:52:54,021 Starting Bokeh server version 3.6.2 (running on Tornado 6.4.2)
2025-02-22 12:52:54,022 User authentication hooks NOT provided (default user enabled)
2025-02-22 12:52:54,023 Bokeh app running at: http://localhost:5006/plot_hr
2025-02-22 12:52:54,023 Starting Bokeh server with process id: 29361
2025-02-22 12:52:54,707 WebSocket connection opened
2025-02-22 12:52:54,708 ServerConnection created
2025-02-22 12:58:37,116 WebSocket connection closed: code=1001, reason=None


To kill the bokeh process: 
 - open terminal
 - write kill -9 <process_id>, where process_id is the process ID of the bokeh server, that can be found in the cell above (Starting Bokeh server with process id: 2553)